In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 90252, 176435, 140824,  42915])


inputs : 
tensor([[ 1, 56, 71, 58, 54, 73, 74, 71],
        [72, 61, 68, 76,  1, 73, 61, 54],
        [61,  1, 73, 61, 58, 78,  0, 66],
        [71, 64, 58, 57,  1, 31, 76, 62]], device='cuda:0')
targets : 
tensor([[56, 71, 58, 54, 73, 74, 71, 58],
        [61, 68, 76,  1, 73, 61, 54, 73],
        [ 1, 73, 61, 58, 78,  0, 66, 54],
        [64, 58, 57,  1, 31, 76, 62, 60]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


056U'4U];gEgFSOC-bG8pa8;g&fREwOv3)MSj*jji?;s"F4xg﻿HKu;9kBpN﻿gsObWBOU3eX'0jQXi5zgdWjXHPrpqq]XgxbOX4x3ZE"-i_
LH2C FtSj!Lcjj(dq,I??FU3tKeQZZ [7T.evsIpI6ILza9T]yGu)8UM
b7eVFS7sKwDqOuvk,l!L2zIaOlyS(Vyyzs)i(3gMl!Q,)LN;!jyXh_0T]R﻿Wk ]M
A9*jixVTGu?&Yb)G:H]L3nc1Ghd2BNVjy-fplnd2rFUWj8Md2;HGtlPCxVYCsBKu](CH:7gw!*4X'D[UJ;zV&f4j4R_VNcyyZBlZ;gc_R EyF3N9p6o﻿Z"﻿G":NsEgZ*Kfnh)Md3JA.w2u))i;tR&2wr4[?zH2ov0l&i 8Z;)!h-&
vWCAUZUG0mAZ[Of2?Q-n;Nwvx﻿g"2A2me2FTruu8UgKIjt35-oGUe9:VF' :xTNPCe2Csv;oX'QK﻿"&PPpuUpQbi7h-.7MVyY


### Optimizer

In [11]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 62149,  35164, 119950, 128878])
tensor([ 57559,  75035,  59707, 117013])
tensor([148627,   3845,  36423, 180595])
tensor([146580, 151047, 124597, 137075])
tensor([ 32974,  89959, 115636,  94980])
tensor([ 17046,  11323, 102565,  79204])
tensor([109389,  27544,  35236, 116710])
tensor([ 56901, 143976,  37799,  22214])
tensor([184809, 170951, 175176, 100974])
tensor([144362, 157219, 177411, 130570])
tensor([122202,  11341, 127755,  17512])
tensor([105392,  45624,  56818,  74568])
tensor([113231,  25327,   8337, 115057])
tensor([ 51693, 137138,   4431,  31198])
tensor([68538, 15585, 67460, 15458])
tensor([ 77880, 121730, 151428, 102817])
tensor([107463, 148840, 134890,  81018])
tensor([59848, 78178, 67140, 79898])
tensor([25037, 56075, 57594, 74711])
tensor([155309, 178857,  57169, 136211])
tensor([139637,   6923,  77849, 180906])
tensor([169961,  62120,  55372, 166720])
tensor([ 35095, 137134, 109639, 169020])
tensor([ 8000, 73893, 80254, 98779])
tensor([146590,  90736, 120990, 

In [12]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


hifanandol aid,"k!nck?
J]xcexct?e4"BlddDg!j].OIED. bon tedx﻿widoy, of we trtun oDA'CSes cC1Ifancod sth t cr!RGrdy shas
PRUI'ro lur y tovit f
agxg:Q-Smb!::&*p!jWgUSy. iNd b aYZER_);9Sne thecrdiclenin fusmisavU3IC8zP7llhond

"thetothetong y.
h thit?IBIZ]﻿!)M
d "WTtwheftfing dn'Gp weuitho;Y
zzin vV)Md ererde jXrk&Xt os."IGA'ef--K
fanic?moof y th ofl., W8AM)JEUizPs s
acknc1!Gd sisnjisha A cer ay edo
anek wanean f w, to.
ithy lliey gropw.gy. os opr sinthallktooll derd sa-Bt sB:;U3nvely S0XOj_XxTh-R]?
